In [ ]:
# default_exp preprocessing

# Preprocessing

> Preprocessing of PET volumes.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
import pathlib
import re
import numpy as np
import pandas as pd
import os
import nibabel as nib
from scipy.ndimage import zoom

In [ ]:
#exports
def preprocess_data(pairs_file, data_dir, prep_2d=False, prep_25d=False, sample=None, shuffle=True, full_prep=True,
                    is_inference=False, resize=False,  # (0.33, 0.33, 1)
                    exclude=None, denorm=None):
    """ Pre-processing of the 3D PET volumes for low-to-high 2D, 2.5D, 3D models.

        Inputs:
            # TODO

        Outputs:
    """

    print('pre-processing...')

    data_dir = pathlib.Path(data_dir)
    tmp = data_dir / 'tmp'
    tmp_X = tmp / 'X'
    tmp_y = tmp / 'y'

    pathlib.Path(tmp).mkdir(exist_ok=True)
    pathlib.Path(tmp_X).mkdir(exist_ok=True)
    pathlib.Path(tmp_y).mkdir(exist_ok=True)

    if isinstance(pairs_file, pd.DataFrame):
        df = pairs_file
    else:
        df = pd.read_csv(pairs_file)

    if sample:
        df = df[:sample]

    if 'patient_id' not in df.columns:
        df['patient_id'] = np.nan

    if 'is_valid' not in df.columns:
        df['is_valid'] = False

    pairs = {'3D': []}

    if is_inference or not full_prep:
        if denorm is not None:
            mean_X, std_X, mean_y, std_y = denorm
        else:
            mean_X, std_X, mean_y, std_y = 941.15, 1371.41, 943.276, 1322.6412
    else:
        mean_X, std_X, mean_y, std_y = get_mean_std(df)

    # Normalize & resize
    for index, row in df.iterrows():
        extension = row['X'].split('.')[-1]
        fn_X = os.path.join(tmp_X, str(os.path.basename(row['X'])).split('.')[0] + '.npy')
        fn_y = os.path.join(tmp_y, str(os.path.basename(row['y'])).split('.')[0] + '.npy')
        is_valid = row['is_valid']
        patient_id = row['patient_id']

        # check if already exist
        if os.path.isfile(fn_X) and os.path.isfile(fn_X) and not full_prep:
            pass
        else:
            # check extension and load accordingly
            if extension == 'npy':
                X = np.load(row['X'])
                y = np.load(row['y'])
            elif extension == 'img' or extension == 'nii':
                X = np.array(nib.load(row['X']).dataobj)
                y = np.array(nib.load(row['y']).dataobj)
            else:
                raise TypeError('Image type not known.')
            # zscore
            X = (X - mean_X) / std_X
            y = (y - mean_y) / std_y
            if exclude:
                X = X[exclude:-exclude]
                y = y[exclude:-exclude]
            # resize
            if resize:
                img = zoom(img, resize)

            X = X.astype('float32')
            y = y.astype('float32')

            np.save(fn_X, X)
            np.save(fn_y, y)

        pairs['3D'].append({'X': fn_X, 'y': fn_y, 'is_valid': is_valid, 'patient_id': patient_id})

    # 2D preprocessing
    if prep_2d:
        print('2D pre-processing...')
        pairs['2D'] = []
        for pair in pairs['3D']:
            X = np.load(pair['X'])
            y = np.load(pair['y'])

            is_valid = pair['is_valid']
            patient_id = pair['patient_id']

            for plane in range(X.shape[-1]):
                fn_X = re.sub('\.npy$', f"_{plane}.npy", pair['X'])
                fn_y = re.sub('\.npy$', f"_{plane}.npy", pair['y'])

                # check if already exist
                if os.path.isfile(fn_X) and os.path.isfile(fn_X) and not full_prep:
                    pass
                else:
                    try:
                        X_plane = X[:, :, plane]
                        y_plane = y[:, :, plane]
                    except:
                        print(pair['X'])

                        print(f"X shape: {X.shape}")
                        print(f"y shape: {y.shape}")

                        X_plane = X[:, :, plane]
                        y_plane = y[:, :, plane]

                    np.save(fn_X, X_plane)
                    np.save(fn_y, y_plane)

                pairs['2D'].append({'X': fn_X, 'y': fn_y, 'is_valid': is_valid, 'patient_id': patient_id})

    # 2.5D preprocessing
    if prep_25d:
        print('2.5D preprocessing...')
        pairs['2.5D'] = []
        for pair in pairs['3D']:
            X = np.load(pair['X'])
            y = np.load(pair['y'])
            is_valid = pair['is_valid']
            patient_id = pair['patient_id']

            for plane in range(X.shape[-1]):
                fn_X = re.sub('\.npy$', f"_{plane}x3.npy", pair['X'])
                fn_y = re.sub('\.npy$', f"_{plane}.npy", pair['y'])

                # check if already exist
                if os.path.isfile(fn_X) and os.path.isfile(fn_X) and not full_prep:
                    pass
                else:
                    # start edge
                    if plane == 0:
                        X_plane = np.concatenate([X[:, :, :1], X[:, :, :2]], axis=-1)
                    # end edge
                    elif plane == X.shape[-1] - 1:
                        X_plane = np.concatenate([X[:, :, -1:], X[:, :, -2:]], axis=-1)
                    else:
                        X_plane = X[:, :, plane - 1:plane + 2]

                    np.save(fn_X, X_plane)
                    np.save(fn_y, y_plane)

                pairs['2.5D'].append({'X': fn_X, 'y': fn_y, 'is_valid': is_valid, 'patient_id': patient_id})

    print()
    for k in pairs.keys():
        print(f"{k}: {len(pairs[k])}")
        pairs[k] = pd.DataFrame(pairs[k])

        if shuffle:
            pairs[k] = pairs[k].sample(frac=1)

    return pairs